In [1]:
import numpy as np
from scipy import ndimage as ndi
#could be accelarate by GPU, just replace the "import numpy" and "scipy" as "import cupy" as below
#import cupy as np
#from cupy import ndimage as ndi
from matplotlib import pyplot as plt
import imageio.v2 as imageio
import glob
import os

# Load Data

In [2]:
%%time
#################################################################
#dir_name is the director to store all the tmp and final results 
data_dir = "demo_data"
#################################################################
#The inital index of data is z,y,x
imagelist = [imageio.imread(file) for file in sorted(glob.glob(os.path.join(data_dir,"*[0-9][0-9][0-9].tif*")))]
data = np.array(imagelist)
del(imagelist)
print("initial data dimensions:", data.shape)
#change the index of data to x,y,z, and change the matrix from view to continous stored array
data = np.ascontiguousarray(np.moveaxis(data,source=[0,2],destination=[2,0]))
print("tunned data dimensions:", data.shape)

initial data dimensions: (361, 548, 364)
tunned data dimensions: (364, 548, 361)
CPU times: user 535 ms, sys: 187 ms, total: 722 ms
Wall time: 719 ms


# generate orientional kernels for convolve operations
ref. https://en.wikipedia.org/wiki/Spherical_coordinate_system  
azimuthal angle $$\phi = tan^{-1}\left(\frac{y}{x}\right)$$
latidute (polar) angle $$\theta = sin^{-1}\left(\frac{\sqrt{x^2+y^2}}{|r|}\right)$$
Cartesian coordinates:
$$x=r\ sin\theta\ cos\phi$$
$$y=r\ sin\theta\ sin\phi$$
$$z=r\ cos\theta\$$

range:$$\phi \subseteq \left [ -\pi/2,\pi/2  \right ] \\
\theta \subseteq \left [ 0,\pi/2  \right ] $$

# Generate line kernels

Time efficient for cup calculations of convolve operations

In [3]:
%%time
def kernel_fun_2(theta, phi, fiber_length = 30):
    half_length = int(fiber_length/2)
    kernel = np.zeros([2*half_length+1,2*half_length+1,2*half_length+1],dtype=np.float32)
    r = np.linspace(-half_length,half_length,1000)
    px = np.rint(r * np.sin(theta) * np.cos(phi) + half_length).astype(int)
    py = np.rint(r * np.sin(theta) * np.sin(phi) + half_length).astype(int)
    pz = np.rint(r * np.cos(theta) + half_length).astype(int)
    kernel[px,py,pz] = 1
    kernel /= kernel.sum()
    return kernel
##############################################################################
#parameters
fiber_length = 30
###############################################################################
#kernel_try_2 = kernel_fun_2(theta = 0.3 , phi = 0.2, fiber_length = fiber_length)
#plot kernel
#import napari
#napari.view_image(kernel_try)

CPU times: total: 0 ns
Wall time: 0 ns


# Calculate the orientations field of 3D volume image

Use line kernels rather than cylinder kernels to save time.

Could be accelate using cupy package (Nvidia GPU needed)

In [4]:
%%time
px = np.zeros_like(data,dtype=np.float16)
py = np.zeros_like(data,dtype=np.float16)
pz = np.zeros_like(data,dtype=np.float16)
correlation = np.zeros_like(data,dtype=np.uint16)

phi_range = np.linspace(-np.pi/2,np.pi/2,36,dtype=np.float32)
theta_range = np.linspace(0,np.pi/2,18,dtype=np.float32)
import time
st = time.time()
for phi in phi_range:
    for theta in theta_range:
        sst = time.time()
        #kernel
        kernel = kernel_fun_2(theta = theta , phi = phi, fiber_length = fiber_length)
        #kernel should be flipped before convovle operation
        kernel = np.flip(kernel,(0,1,2))
        #
        correlation_tmp = ndi.convolve(data, kernel, mode = 'constant', cval = 0.0 )
        update_mask = correlation_tmp>correlation
        correlation[update_mask] = correlation_tmp[update_mask]
        px[update_mask] = np.sin(theta) * np.cos(phi)
        py[update_mask] = np.sin(theta) * np.sin(phi)
        pz[update_mask] = np.cos(theta)
        print("phi-->",phi," theta-->",theta," total_time: ", time.time()-st, "s"," step_time: ", time.time()-sst, "s")
np.save(os.path.join(data_dir,"px.npy"),px)
np.save(os.path.join(data_dir,"py.npy"),py)
np.save(os.path.join(data_dir,"pz.npy"),pz)
np.save(os.path.join(data_dir,"correlation.npy"),correlation)

phi--> -1.5707964  theta--> 0.0  total_time:  6.051610708236694 s  step_time:  6.051610708236694 s
phi--> -1.5707964  theta--> 0.09239978  total_time:  12.61124849319458 s  step_time:  6.558640241622925 s
phi--> -1.5707964  theta--> 0.18479957  total_time:  19.58293581008911 s  step_time:  6.97068452835083 s
phi--> -1.5707964  theta--> 0.27719936  total_time:  26.563611268997192 s  step_time:  6.980675458908081 s
phi--> -1.5707964  theta--> 0.36959913  total_time:  33.542296171188354 s  step_time:  6.978684902191162 s
phi--> -1.5707964  theta--> 0.4619989  total_time:  41.22814059257507 s  step_time:  7.684844732284546 s
phi--> -1.5707964  theta--> 0.5543987  total_time:  48.99675250053406 s  step_time:  7.767611742019653 s
phi--> -1.5707964  theta--> 0.6467985  total_time:  56.91953110694885 s  step_time:  7.922778606414795 s
phi--> -1.5707964  theta--> 0.73919827  total_time:  64.87331175804138 s  step_time:  7.953780651092529 s
phi--> -1.5707964  theta--> 0.83159804  total_time:  72